In [5]:
from time import sleep

import numpy as np
import plotly.graph_objects as go
from numpy._typing import NDArray
from numpy.testing import assert_equal
from tqdm import trange
from zema_emc_annotated.dataset import ZeMASamples  # type: ignore[import]

from lp_nn_robustness_verification.data_acquisition.activation_functions import (
    Sigmoid,
)
from lp_nn_robustness_verification.data_acquisition.generate_nn_params import (
    construct_out_features_counts,
    generate_weights_and_biases,
)
from lp_nn_robustness_verification.data_acquisition.uncertain_inputs import (
    UncertainInputs,
)
from lp_nn_robustness_verification.data_types import (
    UncertainArray,
    ValidCombinationForZeMA,
)
from lp_nn_robustness_verification.linear_program import RobustnessVerification
from lp_nn_robustness_verification.pre_processing import LinearInclusion

## Preparations

Set up shared Plotly layout parameters.

In [6]:
bg_color = "#1E1F22"
shared_layout = dict(
    paper_bgcolor=bg_color,
    font=dict(
        size=20,
    ),
    xaxis_title="1st component",
    yaxis_title="2nd component",
    autosize=False,
    width=1300,
    height=1300,
    showlegend=True,
    legend=dict(
        yanchor="top",
        y=1.05,
        xanchor="left",
        x=0.01,
        font=dict(
            size=30,
        ),
    ),
    plot_bgcolor=bg_color,
)
shared_ratio = dict(
    scaleanchor="x",
    scaleratio=1,
)
shared_title_params = dict(
    y=0.9,
    x=0.5,
    xanchor="center",
    yanchor="top",
)

## Run batch of examples with varying random seed

valid seeds are:

- ValidCombinationForZeMA(size_scaler=1, depth=1): 2
- ValidCombinationForZeMA(size_scaler=1, depth=3): 2123
- ValidCombinationForZeMA(size_scaler=1, depth=5): 80986
- ValidCombinationForZeMA(size_scaler=1, depth=8): 80986
- ValidCombinationForZeMA(size_scaler=10, depth=1): 0

In [12]:
valid_seeds: dict[ValidCombinationForZeMA, int] = {}
size_scalers = [10]
depths = [1]
for size_scaler in size_scalers:
    zema_data = ZeMASamples(1, size_scaler=size_scaler, normalize=True)
    uncertain_inputs = UncertainInputs(
        UncertainArray(zema_data.values[0], zema_data.uncertainties[0])
    )
    weights_and_biases = generate_weights_and_biases(
        len(zema_data.values[0]),
        construct_out_features_counts(len(zema_data.values[0])),
    )
    for depth in depths:
        print(
            f"Trying to find valid seed for {ValidCombinationForZeMA(size_scaler, depth)}"
        )
        for seed in trange(1):
            linear_inclusion = LinearInclusion(
                uncertain_inputs,
                Sigmoid,
                generate_weights_and_biases(
                    len(uncertain_inputs.values),
                    construct_out_features_counts(
                        len(uncertain_inputs.values), depth=depth
                    ),
                    seed,
                ),
            )
            optimization = RobustnessVerification(linear_inclusion)
            optimization.model.hideOutput()
            optimization.solve()
            if optimization.model.getSols():
                valid_seeds[ValidCombinationForZeMA(size_scaler, depth)] = seed
                break
sleep(0.5)
print(f"valid seeds: {valid_seeds}")

Trying to find valid seed for ValidCombinationForZeMA(size_scaler=10, depth=1)


  0%|          | 0/1 [00:00<?, ?it/s]


valid seeds: {ValidCombinationForZeMA(size_scaler=10, depth=1): 0}


## Solution to the linear optimization problem

In [21]:
optimization = RobustnessVerification(linear_inclusion)
solution: str = optimization.solve()
print(solution)

['x_0^(0): 0.0', 'x_1^(0): 0.0', 'x_2^(0): 0.0', 'x_3^(0): 6.0', 'x_4^(0): -6.881462621685058', 'x_5^(0): -0.2215355540367431', 'x_6^(0): -0.413697231651273', 'x_7^(0): 0.6995292826197321', 'x_8^(0): -18.0', 'x_9^(0): 0.2974883214727835', 'x_10^(0): 41.0', 'x_0^(1): 0.6625012289928432', 'x_1^(1): 0.9848070724146616', 'inf r_0^(1): -0.0', 'sup r_0^(1): -0.0', 'inf r_1^(1): -0.0', 'sup r_1^(1): -0.0', '\n']
presolving:
(round 1, fast)       5 del vars, 5 del conss, 0 add conss, 5 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 2, fast)       9 del vars, 7 del conss, 0 add conss, 5 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
presolving (3 rounds: 3 fast, 1 medium, 1 exhaustive):
 20 deleted vars, 7 deleted constraints, 0 added constraints, 5 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 0 cliques
transformed 1/1 original solutions to the transformed problem space
Presolving Time: 0.00

SCIP St

### Extract and prepare linear inclusion parameters

In [111]:
def _construct_edge_idxs(
    x_coords: NDArray[np.double], y_coords: NDArray[np.double]
) -> NDArray[np.double]:
    """Build two dimensional vector of x and y coordinates along the edges of input vectors

    The edges will be parallel to x and y axes.
    """
    assert_equal(len(x_coords), len(y_coords))
    return np.vstack(
        (
            np.concatenate(
                (
                    x_coords[x_coords == x_coords.min()].repeat(len(y_coords)),
                    x_coords[
                        np.logical_and(
                            x_coords != x_coords.min(), x_coords != x_coords.max()
                        )
                    ],
                    x_coords[x_coords == x_coords.max()].repeat(len(y_coords)),
                    x_coords[
                        np.logical_and(
                            x_coords != x_coords.min(), x_coords != x_coords.max()
                        )
                    ],
                    x_coords[0:1],
                )
            ),
            np.concatenate(
                (
                    y_coords,
                    np.repeat([y_coords.max()], len(x_coords) - 2),
                    y_coords[-1::-1],
                    np.repeat([y_coords.min()], len(x_coords) - 2),
                    y_coords[0:1],
                )
            ),
        )
    )

In [112]:
theta = dict()
theta_0_0 = linear_inclusion.theta[0][0][0]
theta_1_0 = linear_inclusion.theta[0][1][0]
theta_0 = _construct_edge_idxs(np.array(theta_0_0), np.array(theta_1_0))
theta_0_1 = linear_inclusion.theta[1][0][0]
theta_1_1 = linear_inclusion.theta[1][1][0]
theta_1 = _construct_edge_idxs(np.array(theta_0_1), np.array(theta_1_1))
z_0_1 = linear_inclusion.z_is[0][0][0]
z_1_1 = linear_inclusion.z_is[0][1][0]
xi_0_1 = linear_inclusion.xi_is[0][0]
xi_1_1 = linear_inclusion.xi_is[0][1]
xi_1 = np.vstack((xi_0_1, xi_1_1))
assert_equal(xi_1.shape, (2, 1))
if optimization.model.getSols():
    lower_r_0_1 = optimization.model.getVal(optimization.r_is["inf", 1, 0])
    lower_r_1_1 = optimization.model.getVal(optimization.r_is["inf", 1, 1])
    upper_r_i_0_1 = optimization.model.getVal(optimization.r_is["sup", 1, 0])
    upper_r_i_1_1 = optimization.model.getVal(optimization.r_is["sup", 1, 1])
else:
    lower_r_0_1 = linear_inclusion.r_is[0][0][0].inf
    lower_r_1_1 = linear_inclusion.r_is[0][1][0].inf
    upper_r_i_0_1 = linear_inclusion.r_is[0][0][0].sup
    upper_r_i_1_1 = linear_inclusion.r_is[0][1][0].sup
lower_r_1 = np.vstack((lower_r_0_1, lower_r_1_1))
assert_equal(lower_r_1.shape, (2, 1))
upper_r_1 = np.vstack((upper_r_i_0_1, upper_r_i_1_1))
assert_equal(upper_r_1.shape, (2, 1))
points_per_dim = 20
points_per_edges = 4 * (points_per_dim - 1) + 1
x_0 = _construct_edge_idxs(
    np.linspace(theta_0_0.inf, theta_0_0.sup, points_per_dim),
    np.linspace(theta_1_0.inf, theta_1_0.sup, points_per_dim),
)
assert_equal(x_0.shape, (2, points_per_edges))
x_1 = _construct_edge_idxs(
    np.linspace(theta_0_1.inf, theta_0_1.sup, points_per_dim),
    np.linspace(theta_1_1.inf, theta_1_1.sup, points_per_dim),
)
assert_equal(x_1.shape, (2, points_per_edges))

In [123]:
linear_inclusion.theta

((interval([-5.1606377858191035, 4.074591920636097]),
  interval([-1.393796995951563, 0.6986749809092526]),
  interval([-2.0962371731224474, 2.2605746102500914]),
  interval([5.8890049816052095, 6.003067220061309]),
  interval([-6.942075231666047, -6.881462621685058]),
  interval([-0.228323333512594, -0.2215355540367431]),
  interval([-0.4261696883354121, -0.413697231651273]),
  interval([0.6995292826197321, 0.721012414191569]),
  interval([-19.350435426208204, -17.656951187566133]),
  interval([0.2974883214727835, 0.2976840607659673]),
  interval([40.08559882909854, 41.31497268860452])),
 (interval([0.9828098716920776, 0.997980117534479]),
  interval([0.565907273333344, 0.8829468095915568])))

In [113]:
fig_edge = go.Figure()
fig_edge.add_trace(
    go.Scatter(x=x_0[0], y=x_0[1], mode="none", name="$\Large{x^{(0)}}$", fill="toself")
)
fig_edge.update_layout(
    title=dict(text="Inputs", **shared_title_params), **shared_layout
)
fig_edge.update_yaxes(**shared_ratio)
fig_edge.show()

In [120]:
linear_inclusion.nn_params.weights[0].shape

(2, 11)

In [119]:
x_0.shape

(2, 77)

In [118]:
linear_inclusion.nn_params.weights[0] @ x_0

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 2 is different from 11)

In [114]:
z_1 = (
    (linear_inclusion.nn_params.weights[0] @ x_0).transpose()
    + linear_inclusion.nn_params.biases[0]
).transpose()
assert z_1.shape == x_0.shape
z_1

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 2 is different from 11)

In [92]:
h_of_z_1 = np.apply_along_axis(linear_inclusion.activation.func, 1, z_1)
assert h_of_z_1.shape == z_1.shape

In [63]:
taylor_approx_1 = linear_inclusion.activation.func(
    xi_1
) + linear_inclusion.activation.deriv(xi_1) * (z_1 - xi_1)
assert taylor_approx_1.shape == z_1.shape
taylor_approx_1.shape, x_1.shape

((2, 77), (2, 77))

In [64]:
taylor_plus_lower_r_1 = taylor_approx_1 + lower_r_1
assert taylor_plus_lower_r_1.shape == taylor_approx_1.shape
lower_linear_1 = np.array(
    [x - taylor for taylor in taylor_plus_lower_r_1.T for x in x_1.T]
).T
assert_equal(lower_linear_1.shape, (2, np.square(points_per_edges)))

In [65]:
a, b = np.array([[1.0, 1.0, 1.0], [1.0, 2.0, 3.0]]), np.array([[2.0, 3.0], [1.0, 2.0]])
a, b, a.shape, b.shape

(array([[1., 1., 1.],
        [1., 2., 3.]]),
 array([[2., 3.],
        [1., 2.]]),
 (2, 3),
 (2, 2))

In [66]:
np.array([taylor - x for taylor in a.T for x in b.T]).T

array([[-1., -2., -1., -2., -1., -2.],
       [ 0., -1.,  1.,  0.,  2.,  1.]])

In [67]:
taylor_plus_upper_r_1 = taylor_approx_1 + upper_r_1
assert taylor_plus_upper_r_1.shape == taylor_approx_1.shape
upper_linear_1 = np.array(
    [x - taylor for taylor in taylor_plus_upper_r_1.T for x in x_1.T]
).T
assert lower_linear_1.shape == (2, np.square(points_per_edges))

## Visualize input region and discretization

In [121]:
fig_inputs = go.Figure()
fig_inputs.add_trace(
    go.Scatter(
        x=x_0[0], y=x_0[1], mode="none", name=r"$\Large{x^{(0)}}$", fill="toself"
    )
)
fig_inputs.add_trace(
    go.Scatter(x=theta_0[0], y=theta_0[1], mode="lines", name=r"$\Large{\Theta^{(0)}}$")
)
fig_inputs.update_layout(
    title=dict(text="Inputs", **shared_title_params), **shared_layout
)
fig_inputs.update_yaxes(**shared_ratio)
fig_inputs.show()

## Visualize first layer's transformations with discretizations

In [35]:
fig_1st_transforms = go.Figure()
fig_1st_transforms.add_trace(
    go.Scatter(
        x=theta_0[0],
        y=theta_0[1],
        mode="lines",
        name=r"$\Large{\Theta^{(0)}}$",
    )
)
fig_1st_transforms.add_trace(
    go.Scatter(x=theta_1[0], y=theta_1[1], mode="lines", name=r"$\Large{\Theta^{(1)}}$")
)
fig_1st_transforms.add_trace(
    go.Scatter(x=z_1[0], y=z_1[1], mode="markers", name=r"$\Large{z^{(1)}}$")
)
fig_1st_transforms.add_trace(
    go.Scatter(
        x=(z_0_1.inf, z_0_1.inf, z_0_1.sup, z_0_1.sup, z_0_1.inf),
        y=(z_1_1.inf, z_1_1.sup, z_1_1.sup, z_1_1.inf, z_1_1.inf),
        mode="lines",
        name=r"$\Large{Z^{(1)}}$",
    )
)
fig_1st_transforms.add_trace(
    go.Scatter(
        x=h_of_z_1[0],
        y=h_of_z_1[1],
        mode="none",
        name=r"$\Large{h (z^{(1)}})$",
        fill="toself",
    )
)
fig_1st_transforms.update_layout(
    title=dict(text="First layer's transformations", **shared_title_params),
    **shared_layout,
)
ytick_vals = np.linspace(z_1[1].min(), z_1[1].max(), 9)
assert ytick_vals[len(ytick_vals) // 2] == xi_1_1
ytick_labels = list(ytick_vals.round(1))
ytick_labels[len(ytick_labels) // 2] = r"$\Large{\xi_1^{(1)}}$"
xtick_vals = np.linspace(z_1[0].min(), z_1[0].max(), 9)
assert xtick_vals[len(xtick_vals) // 2] == xi_0_1
xtick_labels = list(xtick_vals.round(1))
xtick_labels[len(xtick_labels) // 2] = r"$\Large{\xi_0^{(1)}}$"
fig_1st_transforms.update_yaxes(
    **shared_ratio, tickvals=ytick_vals, ticktext=ytick_labels
)
fig_1st_transforms.update_xaxes(
    **shared_ratio, tickvals=xtick_vals, ticktext=xtick_labels
)
fig_1st_transforms.show()

In [41]:
fig_linear_inclusion = go.Figure()
fig_linear_inclusion.add_trace(
    go.Scatter(
        x=theta_0[0],
        y=theta_0[1],
        mode="lines",
        name=r"$\Large{\Theta^{(0)}}$",
    )
)
fig_linear_inclusion.add_trace(
    go.Scatter(
        x=theta_1[0],
        y=theta_1[1],
        mode="lines",
        name=r"$\Large{\Theta^{(1)}}$",
    )
)
fig_linear_inclusion.add_trace(
    go.Scatter(x=z_1[0], y=z_1[1], mode="markers", name=r"$\Large{z^{(1)}}$")
)
fig_linear_inclusion.add_trace(
    go.Scatter(
        x=lower_linear_1[0],
        y=lower_linear_1[1],
        mode="none",
        name="lower bound",
        fill="toself",
    )
)
fig_linear_inclusion.add_trace(
    go.Scatter(
        x=upper_linear_1[0][upper_linear_1[0] >= 0],
        y=upper_linear_1[1][upper_linear_1[0] >= 0],
        mode="none",
        name="upper bound",
        fill="toself",
    )
)
fig_linear_inclusion.add_trace(
    go.Scatter(
        x=(z_0_1.inf, z_0_1.inf, z_0_1.sup, z_0_1.sup, z_0_1.inf),
        y=(z_1_1.inf, z_1_1.sup, z_1_1.sup, z_1_1.inf, z_1_1.inf),
        mode="lines",
        name=r"$\Large{Z^{(1)}}$",
    )
)
fig_linear_inclusion.update_layout(
    title=dict(text="First layer's linear inclusion", **shared_title_params),
    **shared_layout,
)
fig_linear_inclusion.update_yaxes(**shared_ratio)
fig_linear_inclusion.show()

## Robustness verification of the ZeMA dataset

In [63]:
from zema_emc_annotated.dataset import ZeMASamples  # type: ignore[import]

from lp_nn_robustness_verification.data_acquisition.activation_functions import Sigmoid
from lp_nn_robustness_verification.data_acquisition.generate_nn_params import (
    construct_out_features_counts,
    generate_weights_and_biases,
)
from lp_nn_robustness_verification.data_acquisition.uncertain_inputs import (
    UncertainInputs,
)
from lp_nn_robustness_verification.data_types import UncertainArray
from lp_nn_robustness_verification.linear_program import RobustnessVerification
from lp_nn_robustness_verification.pre_processing import LinearInclusion

zema_data = ZeMASamples(100, size_scaler=1, normalize=True)
nn_params = generate_weights_and_biases(
    len(zema_data.values[0]), construct_out_features_counts(len(zema_data.values[0]))
)
for (values, uncertainties) in zip(zema_data.values, zema_data.uncertainties):
    linear_inclusion = LinearInclusion(
        uncertain_inputs=UncertainInputs(UncertainArray(values, uncertainties)),
        activation=Sigmoid,
        nn_params=nn_params,
    )
    optimization_model = RobustnessVerification(linear_inclusion)
    print(optimization_model.solve())

Nonepresolving:

presolving (1 rounds: 1 fast, 0 medium, 0 exhaustive):
 5 deleted vars, 4 deleted constraints, 0 added constraints, 8 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 0 cliques
presolving detected infeasibility
Presolving Time: 0.00

SCIP Status        : problem is solved [infeasible]
Solving Time (sec) : 0.00
Solving Nodes      : 0
Primal Bound       : -1.00000000000000e+20 (objective limit, 0 solutions)
Dual Bound         : -1.00000000000000e+20
Gap                : 0.00 %
None
presolving:
presolving (1 rounds: 1 fast, 0 medium, 0 exhaustive):
 5 deleted vars, 4 deleted constraints, 0 added constraints, 8 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 0 cliques
presolving detected infeasibility
Presolving Time: 0.00

SCIP Status        : problem is solved [infeasible]
Solving Time (sec) : 0.00
Solving Nodes      : 0
Primal Bound       : -1.00000000000000e+20 (objective limit, 0 soluti